От www.kaggle.com решаваме да изтеглим и да разгледаме dataset-а "mushroom-classification". Това е пример за лейбълнати данни, където може да бъде направена класификация, защото label-ът се състои от 2 състояния - дали гъбата е ядлива или е отровна. Затова не може да става дума за регресия. Нека първо си заредим dataset-а, а също и необходимите библиотеки.

In [1]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [9]:
train = pd.read_csv('data/mushroom-classification/mushrooms.csv')

В този случай колоната "class" ще бъде използвана за label на нашия dataset. Нека да видим какви други колони има:

In [8]:
print(dataset.columns)

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')


Сега ще се опитаме да разделим данните на тренировъчен и на тестов сет, като ще се опитаме да обучим Линейна или Логистична регресия да различават коя гъба е ядлива и коя - отровна.

In [10]:
from sklearn.model_selection import train_test_split

train_no_class = train.drop(labels=['class'], axis=1)
train_class = train['class']

X_train, X_test, y_train, y_test = train_test_split(train_no_class, 
                                                    train_class, 
                                                    test_size=0.3, 
                                                    random_state=4330)

Колоната "class" ще служи за label на нашите данни.
Нека сега да видим колко на брой данни имаме за трениране и след това за тестване:

In [13]:
print("train.shape: ", X_train.shape)
print("train.y: ", y_train.shape)

train.shape:  (5686, 22)
train.y:  (5686,)


In [14]:
print("test.shape: ", X_test.shape)
print("test.y: ", y_test.shape)

test.shape:  (2438, 22)
test.y:  (2438,)


Нека да натренираме една линейна регресия:

In [16]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

ValueError: could not convert string to float: 'g'

Явно има стойности, които са различни от число. В лекция номер 4 видяхме как се използват LabelEncoder и OneHotEncoder, за да променят string-овите данни в числови. Там обаче label-ът беше сумата на апартамента, който беше число, и съответно нямаше нужда да се променя. Тук обаче не само част от данните са символни, ами всичките - включително и първата колона, която в случая е нашият label (дали гъбата е ядлива или отровна). Затова може би ще бъде по-добре да променим целият дейтасет и отначало да го разделим на тренировъчен и тестови.

In [65]:
from sklearn.preprocessing import LabelEncoder

train = train.fillna("")
encoders = {col: LabelEncoder().fit(train[col]) for col in train.columns}

print(encoders)

{'class': LabelEncoder(), 'cap-shape': LabelEncoder(), 'cap-surface': LabelEncoder(), 'cap-color': LabelEncoder(), 'bruises': LabelEncoder(), 'odor': LabelEncoder(), 'gill-attachment': LabelEncoder(), 'gill-spacing': LabelEncoder(), 'gill-size': LabelEncoder(), 'gill-color': LabelEncoder(), 'stalk-shape': LabelEncoder(), 'stalk-root': LabelEncoder(), 'stalk-surface-above-ring': LabelEncoder(), 'stalk-surface-below-ring': LabelEncoder(), 'stalk-color-above-ring': LabelEncoder(), 'stalk-color-below-ring': LabelEncoder(), 'veil-type': LabelEncoder(), 'veil-color': LabelEncoder(), 'ring-number': LabelEncoder(), 'ring-type': LabelEncoder(), 'spore-print-color': LabelEncoder(), 'population': LabelEncoder(), 'habitat': LabelEncoder()}


In [66]:
print(encoders['class'].classes_)

['e' 'p']


In [67]:
def encode_categorical(data, columns, encoders):
    data = data.fillna("")
    return pd.DataFrame({col: encoders[col].transform(data[col]) for col in columns},
                        index = data.index)

In [71]:
train_encoded = encode_categorical(train, train.columns, encoders)

pd.options.display.max_columns=12
train_encoded.head(8)

,bruises,cap-color,cap-shape,cap-surface,class,gill-attachment,...,stalk-root,stalk-shape,stalk-surface-above-ring,stalk-surface-below-ring,veil-color,veil-type
0,1,4,5,2,1,1,...,3,0,2,2,2,0
1,1,9,5,2,0,1,...,2,0,2,2,2,0
2,1,8,0,2,0,1,...,2,0,2,2,2,0
3,1,8,5,3,1,1,...,3,0,2,2,2,0
4,0,3,5,2,0,1,...,3,1,2,2,2,0
5,1,9,5,3,0,1,...,2,0,2,2,2,0
6,1,8,0,2,0,1,...,2,0,2,2,2,0
7,1,8,0,3,0,1,...,2,0,2,2,2,0


Сега след като сме преработили всички данни, идва моментът да ги разделим на тестови и тренировъчен сет отново.

In [75]:
from sklearn.model_selection import train_test_split

train_no_class = train_encoded.drop(labels=['class'], axis=1)
train_class = train_encoded['class']

X_train, X_test, y_train, y_test = train_test_split(train_no_class, 
                                                    train_class, 
                                                    test_size=0.3, 
                                                    random_state=4330)

Нека отново да опитаме да тренираме линейната регресия, дефинирана по-рано...

In [76]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [80]:
print(regressor.score(X_train, y_train))
print(regressor.score(X_test, y_test))

0.747183701258
0.733342718245


Бих искал да начертая точките в пространството, за да видя как са разположени. По този начин бих могъл да разбера дали за тях е подходящо да се приложи линейна регресия (тоест точките в пространството изглеждат сякаш лежат на една права) или някой по-сложен нелинеен модел. За съжаление знам как това става само за точки, които са в двоичното пространство, тоест са от сорта на (x,y), но този дейтасет далеч не е такъв. Ще опитам да подобря резултата чрез регуляризация от тип L2.

In [97]:
from sklearn.linear_model import Lasso, Ridge

ridge = Ridge(alpha = 0.01)
ridge.fit(X_train, y_train)

print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.747183700665
0.733342658853
